# 字符级语言模型 - 恐龙乐园

欢迎来到恐龙岛！🦖  
在6500万年前，恐龙曾经存在，而在本次课程中，它们又回来了。  
你负责一个特殊任务：生物学顶尖研究人员正在创造新的恐龙品种并将它们带回地球，而你的工作是为这些恐龙**命名**。  
如果恐龙不喜欢它的名字，它可能会发狂，所以请慎重选择！

<table>
<td>
<img src="images/dino.jpg" style="width:250;height:300px;">
</td>
</table>

---

幸运的是，你已经学习了一些深度学习知识，可以用它来完成这个任务。  
你的助手已经收集了所有能找到的恐龙名字，并将它们整理到这个 [数据集](dinos.txt) 中。（可以点击链接查看数据内容）  
为了创造新的恐龙名字，你将构建一个**字符级语言模型（character-level language model）**。  
你的算法将学习不同名字的模式，并随机生成新的名字。希望这个算法能帮你避免恐龙的愤怒！  

---

通过完成本课程，你将学到：

- 如何将文本数据存储并用于 RNN 处理；
- 如何生成数据，通过在每个时间步采样预测结果并将其传递给下一个 RNN 单元；
- 如何构建**字符级文本生成循环神经网络**；
- 为什么梯度裁剪（gradient clipping）很重要。

---

我们将从加载提供的 `rnn_utils` 函数开始。  
你可以使用诸如 `rnn_forward` 和 `rnn_backward` 等函数，它们与前一个作业中你实现的功能相同。


In [2]:
# 导入 NumPy 库，用于科学计算和矩阵操作
import numpy as np  

# 从 utils 模块导入所有函数和变量
# utils.py 是你自己写的工具模块，可能包含数据处理、绘图、损失函数等自定义函数
from utils import *  

# 导入 Python 内置的 random 模块，用于生成随机数、随机选择等操作
import random  

# 从 random 模块中单独导入 shuffle 函数
# shuffle 可以将列表中的元素随机打乱顺序
from random import shuffle  


## 1 - 问题描述

### 1.1 - 数据集与预处理

运行下面的单元格，读取恐龙名字数据集，  
创建一个唯一字符列表（例如 a-z），并计算数据集大小以及词汇表大小。


In [3]:
# 打开文本文件 'dinos.txt'，以只读模式 'r' 打开，并读取文件内容
# read() 会将整个文件内容读取成一个字符串
data = open('dinos.txt', 'r').read()  

# 将文本全部转换为小写字母，保证大小写一致性
# 这样可以减少模型需要学习的字符种类
data = data.lower()  

# 使用 set() 获取文本中所有唯一字符，再用 list() 转换为列表
# chars 列表中每个元素是文本中出现过的一个字符（无重复）
chars = list(set(data))  

# 计算文本总字符数，即文本长度
data_size = len(data)  

# 计算文本中唯一字符的数量，即字典大小
vocab_size = len(chars)  

# 打印文本总字符数和唯一字符数
# %d 是格式化占位符，data_size 和 vocab_size 将分别替换
print('There are %d total characters and %d unique characters in your data.' % (data_size, vocab_size))


There are 19909 total characters and 27 unique characters in your data.


这些字符包括 a-z（共 26 个字符）以及换行符 `\n`，  
在本课程中，它的作用类似于 `<EOS>`（句子结束）标记，只不过这里表示**恐龙名字的结束**，而不是句子的结束。

在下面的单元格中，我们创建了一个 Python 字典（哈希表）：
- `char_to_ix`：将每个字符映射为索引 0-26；
- `ix_to_char`：将每个索引映射回对应字符。

这样，你就可以根据 softmax 输出的概率分布确定每个索引对应的字符。


In [4]:
# 创建字符到索引的映射字典
# enumerate() 会返回一个索引 i 和字符 ch
# sorted(chars) 对字符列表进行排序，这样每次运行结果一致
# 最终得到字典 char_to_ix，key 为字符，value 为对应的索引
char_to_ix = { ch:i for i,ch in enumerate(sorted(chars)) }

# 创建索引到字符的映射字典
# 作用与上面的字典相反
# key 为索引 i，value 为字符 ch
ix_to_char = { i:ch for i,ch in enumerate(sorted(chars)) }

# 打印索引到字符的映射字典，便于查看结果
# 例如：{0: ' ', 1: 'a', 2: 'b', ...}
print(ix_to_char)


{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


### 1.2 - 模型概览

你的模型将具有以下结构：

- 初始化参数  
- 运行优化循环：
  - 前向传播计算损失函数  
  - 反向传播计算损失函数关于参数的梯度  
  - 对梯度进行裁剪（gradient clipping），以避免梯度爆炸  
  - 使用梯度更新参数（梯度下降更新规则）  
- 返回训练得到的参数

<img src="images/rnn.png" style="width:450;height:300px;">
<caption><center> **图 1**：循环神经网络（RNN），与之前 “Building a RNN - Step by Step” 中构建的网络类似。 </center></caption>

在每个时间步，RNN 尝试预测下一个字符，给定前面的字符。  
- 数据集 $X = (x^{\langle 1 \rangle}, x^{\langle 2 \rangle}, ..., x^{\langle T_x \rangle})$ 是训练集中字符的列表；  
- 数据集 $Y = (y^{\langle 1 \rangle}, y^{\langle 2 \rangle}, ..., y^{\langle T_x \rangle})$ 满足：在每个时间步 $t$，有 $y^{\langle t \rangle} = x^{\langle t+1 \rangle}$。


## 2 - 模型的构建模块

在本部分，你将实现整个模型的两个重要模块：

- **梯度裁剪（Gradient Clipping）**：用于避免梯度爆炸  
- **采样（Sampling）**：用于生成字符的技术

随后，你将使用这两个函数来构建完整的模型。


### 2.1 - 在优化循环中裁剪梯度

在本节中，你将实现 `clip` 函数，并在优化循环中调用它。  
回顾一下，整个循环结构通常包括：前向传播 → 损失计算 → 反向传播 → 参数更新。  
在更新参数之前，需要进行**梯度裁剪**，确保梯度不会“爆炸”，即取值过大。

---

在练习中，你将实现 `clip` 函数：
- 输入：一个梯度字典 `gradients`  
- 输出：裁剪后的梯度字典  

裁剪方法：**逐元素裁剪**（element-wise clipping）  
- 给定一个阈值 `maxValue`（例如 10）  
- 若梯度分量大于 10，则设为 10  
- 若梯度分量小于 -10，则设为 -10  
- 若梯度分量在 [-10, 10] 内，则保持不变

<img src="images/clip.png" style="width:400;height:150px;">
<caption><center> **图 2**：有无梯度裁剪的梯度下降示意图，当网络出现轻微“梯度爆炸”问题时。 </center></caption>

---

**练习**：实现 `clip` 函数，返回裁剪后的梯度字典。  
可以参考此 [提示](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.clip.html)。


In [7]:
def clip(gradients, maxValue):
    '''
    对梯度进行裁剪（Clipping），防止梯度爆炸。
    
    参数：
    gradients -- 包含各个梯度的字典，常见键包括:
                 "dWaa"  -> 隐藏到隐藏权重矩阵的梯度
                 "dWax"  -> 输入到隐藏权重矩阵的梯度
                 "dWya"  -> 隐藏到输出权重矩阵的梯度
                 "db"    -> 隐藏层偏置的梯度
                 "dby"   -> 输出层偏置的梯度
    maxValue -- 最大梯度阈值，梯度会被限制在 [-maxValue, maxValue] 范围内
    
    返回：
    gradients -- 裁剪后的梯度字典
    '''

    # 从字典中提取各个梯度矩阵
    dWaa, dWax, dWya, db, dby = gradients['dWaa'], gradients['dWax'], gradients['dWya'], gradients['db'], gradients['dby']
   
    # 对每一个梯度进行裁剪操作
    # np.clip(gradient, min, max, out=gradient) 会将 gradient 中大于 max 的值设为 max，
    # 小于 min 的值设为 min，并直接修改原矩阵（out=gradient）
    for gradient in [dWaa, dWax, dWya, db, dby]:
        np.clip(gradient, -maxValue, maxValue, out=gradient)
    
    # 将裁剪后的梯度重新打包成字典，便于返回
    gradients = {"dWaa": dWaa, "dWax": dWax, "dWya": dWya, "db": db, "dby": dby}
    
    return gradients


In [8]:
# 设置随机种子，保证每次生成的随机数一致，便于实验可复现
np.random.seed(3)

# 随机生成梯度矩阵并放大10倍，模拟梯度可能过大的情况
dWax = np.random.randn(5,3) * 10  # 输入到隐藏层权重的梯度，5行3列
dWaa = np.random.randn(5,5) * 10  # 隐藏到隐藏层权重的梯度，5行5列
dWya = np.random.randn(2,5) * 10  # 隐藏到输出层权重的梯度，2行5列
db = np.random.randn(5,1) * 10    # 隐藏层偏置的梯度，5行1列
dby = np.random.randn(2,1) * 10   # 输出层偏置的梯度，2行1列

# 将所有梯度打包成字典，便于函数传参
gradients = {"dWax": dWax, "dWaa": dWaa, "dWya": dWya, "db": db, "dby": dby}

# 调用clip函数，对梯度进行裁剪，阈值设置为10
# 超过10的值会被限制为10，小于-10的值会被限制为-10
gradients = clip(gradients, 10)

# 输出裁剪后的梯度矩阵中指定元素，观察裁剪效果
print("gradients[\"dWaa\"][1][2] =", gradients["dWaa"][1][2])  # 查看dWaa第2行第3列
print("gradients[\"dWax\"][3][1] =", gradients["dWax"][3][1])  # 查看dWax第4行第2列
print("gradients[\"dWya\"][1][2] =", gradients["dWya"][1][2])  # 查看dWya第2行第3列
print("gradients[\"db\"][4] =", gradients["db"][4])            # 查看db第5行
print("gradients[\"dby\"][1] =", gradients["dby"][1])          # 查看dby第2行


gradients["dWaa"][1][2] = 10.0
gradients["dWax"][3][1] = -10.0
gradients["dWya"][1][2] = 0.2971381536101662
gradients["db"][4] = [10.]
gradients["dby"][1] = [8.45833407]


** Expected output:**

<table>
<tr>
    <td> 
    **gradients["dWaa"][1][2] **
    </td>
    <td> 
    10.0
    </td>
</tr>

<tr>
    <td> 
    **gradients["dWax"][3][1]**
    </td>
    <td> 
    -10.0
    </td>
    </td>
</tr>
<tr>
    <td> 
    **gradients["dWya"][1][2]**
    </td>
    <td> 
0.29713815361
    </td>
</tr>
<tr>
    <td> 
    **gradients["db"][4]**
    </td>
    <td> 
[ 10.]
    </td>
</tr>
<tr>
    <td> 
    **gradients["dby"][1]**
    </td>
    <td> 
[ 8.45833407]
    </td>
</tr>

</table>

### 2.2 - 采样（Sampling）

现在假设你的模型已经训练完成，你希望生成新的文本（字符）。  
生成过程如下图所示：

<img src="images/dinos3.png" style="width:500;height:300px;">
<caption><center> **图 3**：假设模型已训练完毕。我们在第一时间步输入 $x^{\langle 1\rangle} = \vec{0}$，网络随后逐步采样生成字符。 </center></caption>

---

**练习**：实现 `sample` 函数来生成字符，包含以下四个步骤：

#### **步骤 1**：初始化输入
- 将网络的第一个输入设为“虚拟输入” $x^{\langle 1 \rangle} = \vec{0}$（全零向量）  
- 将隐藏状态初始化为 $a^{\langle 0 \rangle} = \vec{0}$

#### **步骤 2**：前向传播一步
计算 $a^{\langle 1 \rangle}$ 和预测 $\hat{y}^{\langle 1 \rangle}$：

$$ a^{\langle t+1 \rangle} = \tanh(W_{ax}  x^{\langle t \rangle } + W_{aa} a^{\langle t \rangle } + b) \tag{1} $$

$$ z^{\langle t + 1 \rangle } = W_{ya}  a^{\langle t + 1 \rangle } + b_y \tag{2} $$

$$ \hat{y}^{\langle t+1 \rangle } = softmax(z^{\langle t + 1 \rangle }) \tag{3} $$

> 注意：$\hat{y}^{\langle t+1 \rangle }$ 是一个概率向量，元素在 0 到 1 之间并且和为 1。  
> $\hat{y}^{\langle t+1 \rangle}_i$ 表示索引为 "i" 的字符是下一个字符的概率。  
> 我们提供了 `softmax()` 函数可直接使用。

#### **步骤 3**：按概率采样
- 根据 $\hat{y}^{\langle t+1 \rangle }$ 的概率分布选择下一个字符的索引。  
- 示例：
```python
np.random.seed(0)
p = np.array([0.1, 0.0, 0.7, 0.2])
index = np.random.choice([0, 1, 2, 3], p = p.ravel())

```
这意味着你将根据概率分布选择 `index`：  
$P(index = 0) = 0.1, P(index = 1) = 0.0, P(index = 2) = 0.7, P(index = 3) = 0.2$。

- **步骤 4**：在 `sample()` 中的最后一步是更新变量 `x`：
  - 当前 `x` 存储的是 $x^{\langle t \rangle }$  
  - 将其更新为 $x^{\langle t + 1 \rangle }$  
  - 使用你预测的字符生成对应的 **one-hot 向量**  
  - 再次进行前向传播，并重复此过程，直到生成 `\n` 字符，表示恐龙名字生成完成。


In [14]:
def sample(parameters, char_to_ix, seed):
    """
    按照RNN输出的概率分布采样字符序列
    
    参数：
    parameters -- 字典，包含RNN的参数Waa, Wax, Wya, by, b
    char_to_ix -- 字典，将字符映射为索引
    seed -- 随机种子，用于保证结果可复现
    
    返回：
    indices -- 一个列表，包含生成字符的索引序列
    """
    
    # 从参数字典中提取RNN参数
    Waa, Wax, Wya, by, b = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['b']
    
    # 词汇表大小，根据输出层偏置维度确定
    vocab_size = by.shape[0]
    
    # 隐藏状态大小，根据Waa列数确定
    n_a = Waa.shape[1]
    
    # 步骤1: 初始化输入字符的独热向量x
    x = np.zeros((vocab_size,1))  # shape = (vocab_size, 1)
    
    # 初始化前一隐藏状态a_prev为零向量
    a_prev = np.zeros((n_a,1))    # shape = (n_a, 1)
    
    # 用于存储采样出的字符索引
    indices = []
    
    # 初始化idx为-1，用作循环控制，直到采样到换行符停止
    idx = -1
    
    # 初始化计数器，用于限制生成的最大字符数
    counter = 0
    
    # 换行符在词汇表中的索引
    newline_character = char_to_ix["\n"]
    
    # 循环直到采样到换行符或生成字符数达到50
    while (idx != newline_character and counter < 50):
        
        # 步骤2：前向传播计算下一个隐藏状态和输出
        # a = tanh(Wax * x + Waa * a_prev + b)
        a = np.tanh(np.dot(Wax, x) + np.dot(Waa, a_prev) + b)
        
        # z = Wya * a + by，用于输出层
        z = np.dot(Wya, a) + by
        
        # y = softmax(z)，得到下一个字符的概率分布
        y = softmax(z)
        
        # 设置随机种子，保证实验可复现
        np.random.seed(counter + seed)
        
        # 步骤3：根据概率分布y随机采样字符索引
        idx = np.random.choice(list(range(vocab_size)), p=y.ravel())
        
        # 将采样索引添加到indices列表中
        indices.append(idx)
        
        # 步骤4: 构造下一时间步输入字符的独热向量
        x = np.zeros((vocab_size,1))  # 先清零
        x[idx] = 1                    # 将采样到的字符位置置为1
        
        # 更新隐藏状态为当前时间步的a
        a_prev = a 
        
        # 更新随机种子和计数器
        seed += 1
        counter += 1

    # 如果达到50个字符仍未采样到换行符，则强制添加换行符
    if (counter == 50):
        indices.append(char_to_ix['\n'])
    
    # 返回生成字符的索引序列
    return indices


In [15]:
# 设置随机种子，使结果可复现
np.random.seed(2)

# 定义生成的字符数量n和隐藏层神经元数量n_a
n, n_a = 20, 100  # n: 输出序列长度(此处未严格用到)，n_a: 隐藏状态维度

# 初始化隐藏状态a0为标准正态分布随机值
a0 = np.random.randn(n_a, 1)  # shape = (n_a, 1)

# 初始化第一个输入字符索引，这里选择i0 = 1，表示词汇表中的第二个字符
i0 = 1  # 第一个字符是 ix_to_char[i0]

# 初始化权重矩阵
# Wax: 输入到隐藏状态的权重，shape = (n_a, vocab_size)
# Waa: 上一隐藏状态到当前隐藏状态的权重，shape = (n_a, n_a)
# Wya: 隐藏状态到输出的权重，shape = (vocab_size, n_a)
Wax, Waa, Wya = np.random.randn(n_a, vocab_size), np.random.randn(n_a, n_a), np.random.randn(vocab_size, n_a)

# 初始化偏置向量
# b: 隐藏层偏置，shape = (n_a, 1)
# by: 输出层偏置，shape = (vocab_size, 1)
b, by = np.random.randn(n_a, 1), np.random.randn(vocab_size, 1)

# 将所有参数存入字典中，便于传递给RNN采样函数
parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "b": b, "by": by}

# 使用sample函数生成字符索引序列
# 参数:
# - parameters: RNN参数字典
# - char_to_ix: 字符到索引映射
# - 0: 随机种子
indexes = sample(parameters, char_to_ix, 0)

# 打印采样结果
print("Sampling:")

# 打印采样到的索引列表
print("list of sampled indices:", indexes)

# 将索引映射回字符，打印采样到的字符序列
print("list of sampled characters:", [ix_to_char[i] for i in indexes])


Sampling:
list of sampled indices: [18, 2, 26, 0]
list of sampled characters: ['r', 'b', 'z', '\n']


** Expected output:**
<table>
<tr>
    <td> 
    **list of sampled indices:**
    </td>
    <td> 
    [18, 2, 26, 0]
    </td>
    </tr><tr>
    <td> 
    **list of sampled characters:**
    </td>
    <td> 
    ['r', 'b', 'z', '\n']
    </td>
    
        
    
</tr>
</table>

## 3 - 构建语言模型

现在是时候构建字符级语言模型，用于文本生成了。

---

### 3.1 - 梯度下降（Gradient Descent）

在本节中，你将实现一个函数，执行**一步随机梯度下降（SGD）**，并在需要时进行梯度裁剪。  
训练过程中，你将逐个样本进行优化，因此优化算法为随机梯度下降。  

回顾一下 RNN 的常用优化循环步骤：

1. 前向传播通过 RNN 计算损失
2. 反向传播计算损失相对于参数的梯度（BPTT）
3. 如有必要，对梯度进行裁剪
4. 使用梯度下降更新参数

**练习**：实现上述优化过程（一次随机梯度下降）。

我们提供了以下函数：

```python
def rnn_forward(X, Y, a_prev, parameters):
    """ 
    执行 RNN 前向传播并计算交叉熵损失。
    返回损失值以及存储用于反向传播的缓存。
    """
    ....
    return loss, cache
    
def rnn_backward(X, Y, parameters, cache):
    """ 
    执行反向传播（时间展开）计算损失相对于参数的梯度。
    同时返回所有隐藏状态。
    """
    ...
    return gradients, a

def update_parameters(parameters, gradients, learning_rate):
    """ 
    使用梯度下降更新参数。
    """
    ...
    return parameters

```

In [20]:
def optimize(X, Y, a_prev, parameters, learning_rate = 0.01):
    """
    执行一次优化步骤（即一轮前向传播 + 反向传播 + 参数更新）。

    参数说明:
    X -- 输入序列（字符索引列表），每个元素是词汇表中某个字符的编号。
    Y -- 目标序列（与X相同，但向左平移一个字符，用于监督训练）。
    a_prev -- 上一个时间步的隐藏状态，shape = (n_a, 1)
    parameters -- 包含RNN参数的字典:
        Wax -- 输入到隐藏层的权重矩阵，shape = (n_a, n_x)
        Waa -- 隐藏层到隐藏层的权重矩阵，shape = (n_a, n_a)
        Wya -- 隐藏层到输出层的权重矩阵，shape = (n_y, n_a)
        b -- 隐藏层偏置项，shape = (n_a, 1)
        by -- 输出层偏置项，shape = (n_y, 1)
    learning_rate -- 学习率（控制每次更新的步长）
    
    返回:
    loss -- 当前时间步的损失值（交叉熵）
    gradients -- 梯度字典，包含以下键：
        dWax, dWaa, dWya, db, dby
    a[len(X)-1] -- 最后一个时间步的隐藏状态 (即下一次迭代的 a_prev)
    """

    # ====== ① 前向传播 ======
    # 输入 X 序列，计算所有时间步的隐藏状态和输出预测，并返回损失值
    # cache 中保存中间结果（隐藏状态、激活值等），供反向传播使用
    loss, cache = rnn_forward(X, Y, a_prev, parameters)
    
    
    # ====== ② 反向传播 ======
    # 根据前向传播缓存的中间变量计算参数的梯度
    gradients, a = rnn_backward(X, Y, parameters, cache)
    
    
    # ====== ③ 梯度裁剪（Gradient Clipping） ======
    # 为防止梯度爆炸，将所有梯度限制在 [-5, 5] 范围内
    gradients = clip(gradients, 5)
    
    
    # ====== ④ 参数更新 ======
    # 使用梯度下降法更新模型参数：
    # W := W - learning_rate * dW
    parameters = update_parameters(parameters, gradients, learning_rate)
    
    
    # ====== ⑤ 返回结果 ======
    # 返回损失值、梯度字典、最后一个隐藏状态
    return loss, gradients, a[len(X)-1]


In [21]:
# ==============================
# 测试 optimize() 函数的运行效果
# ==============================

# 固定随机数种子，保证每次运行结果一致，便于调试和对比
np.random.seed(1)

# 定义词汇表大小（vocab_size = 27）和隐藏层单元数量（n_a = 100）
# 词汇表大小 = 26个英文字母 + '\n'（换行符）
vocab_size, n_a = 27, 100

# 初始化上一个时间步的隐藏状态 a_prev，shape = (n_a, 1)
# 表示RNN在当前样本输入前的内部记忆状态
a_prev = np.random.randn(n_a, 1)


# 随机初始化RNN参数：
# Wax: 输入到隐藏层的权重矩阵，shape = (n_a, vocab_size)
Wax = np.random.randn(n_a, vocab_size)

# Waa: 隐藏层到隐藏层的权重矩阵，shape = (n_a, n_a)
Waa = np.random.randn(n_a, n_a)

# Wya: 隐藏层到输出层的权重矩阵，shape = (vocab_size, n_a)
Wya = np.random.randn(vocab_size, n_a)

# b: 隐藏层的偏置项，shape = (n_a, 1)
b = np.random.randn(n_a, 1)

# by: 输出层的偏置项，shape = (vocab_size, 1)
by = np.random.randn(vocab_size, 1)


# 将所有参数存入一个字典，方便在函数中统一传递
parameters = {
    "Wax": Wax,
    "Waa": Waa,
    "Wya": Wya,
    "b": b,
    "by": by
}


# 定义一个样本的输入序列 X 和输出序列 Y
# X 是一个整数列表，每个整数代表一个字符的索引
X = [12, 3, 5, 11, 22, 3]  # 输入字符索引序列
Y = [4, 14, 11, 22, 25, 26]  # 输出目标字符索引序列（即X向左平移一位）


# 调用 optimize() 执行一次前向 + 反向 + 参数更新
# learning_rate = 0.01 表示学习率为 0.01
loss, gradients, a_last = optimize(X, Y, a_prev, parameters, learning_rate = 0.01)


# 输出当前样本的损失值
print("Loss =", loss)


# 以下输出用于梯度检查（验证反向传播计算是否正确）
# 打印 Waa 梯度矩阵中第 [1][2] 元素的值
print("gradients[\"dWaa\"][1][2] =", gradients["dWaa"][1][2])

# 找出 dWax 矩阵中数值最大的元素所在位置（用于检查数值分布）
print("np.argmax(gradients[\"dWax\"]) =", np.argmax(gradients["dWax"]))

# 打印 dWya 矩阵中第 [1][2] 元素的值
print("gradients[\"dWya\"][1][2] =", gradients["dWya"][1][2])

# 打印 db 向量的第 4 个元素（索引为4）
print("gradients[\"db\"][4] =", gradients["db"][4])

# 打印 dby 向量的第 1 个元素（索引为1）
print("gradients[\"dby\"][1] =", gradients["dby"][1])

# 打印最后一个时间步的隐藏状态 a_last 的第 4 个元素（索引为4）
print("a_last[4] =", a_last[4])


Loss = 126.50397572165369
gradients["dWaa"][1][2] = 0.1947093153471991
np.argmax(gradients["dWax"]) = 93
gradients["dWya"][1][2] = -0.007773876032003623
gradients["db"][4] = [-0.06809825]
gradients["dby"][1] = [0.01538192]
a_last[4] = [-1.]


** Expected output:**

<table>


<tr>
    <td> 
    **Loss **
    </td>
    <td> 
    126.503975722
    </td>
</tr>
<tr>
    <td> 
    **gradients["dWaa"][1][2]**
    </td>
    <td> 
    0.194709315347
    </td>
<tr>
    <td> 
    **np.argmax(gradients["dWax"])**
    </td>
    <td> 93
    </td>
</tr>
<tr>
    <td> 
    **gradients["dWya"][1][2]**
    </td>
    <td> -0.007773876032
    </td>
</tr>
<tr>
    <td> 
    **gradients["db"][4]**
    </td>
    <td> [-0.06809825]
    </td>
</tr>
<tr>
    <td> 
    **gradients["dby"][1]**
    </td>
    <td>[ 0.01538192]
    </td>
</tr>
<tr>
    <td> 
    **a_last[4]**
    </td>
    <td> [-1.]
    </td>
</tr>

</table>

### 3.2 - 模型训练（Training the model）


给定恐龙名字数据集，我们将数据集的每一行（一个名字）作为一个训练样本。  
在随机梯度下降的每 100 步，你将随机采样 10 个名字来查看算法效果。  
记得打乱数据集顺序，使随机梯度下降可以以随机顺序访问样本。

**练习**：按照说明实现 `model()` 函数。  
当 `examples[index]` 包含一个恐龙名字（字符串）时，你可以按如下方式创建一个训练样本 `(X, Y)`：

```python
index = j % len(examples)
X = [None] + [char_to_ix[ch] for ch in examples[index]] 
Y = X[1:] + [char_to_ix["\n"]]

```
注意这里使用：`index = j % len(examples)`，其中 `j = 1....num_iterations`，  
确保 `examples[index]` 总是有效（`index` 小于 `len(examples)`）。  

`X` 的第一个元素为 `None`，在 `rnn_forward()` 中会被解释为 $x^{\langle 0 \rangle} = \vec{0}$。  
此外，这样可以确保 `Y` 与 `X` 相同，但整体左移一个时间步，并在末尾添加 `"\n"`，表示恐龙名字结束。


In [24]:
# ==============================
# 训练恐龙名字生成模型的主函数
# ==============================
def model(data, ix_to_char, char_to_ix, num_iterations = 35000, n_a = 50, dino_names = 7, vocab_size = 27):
    """
    训练模型并生成恐龙名字。
    
    参数说明:
    data -- 文本语料库（所有恐龙名字）
    ix_to_char -- 从索引到字符的映射字典
    char_to_ix -- 从字符到索引的映射字典
    num_iterations -- 训练迭代次数（默认35000）
    n_a -- RNN隐藏层神经元数量（默认50）
    dino_names -- 每2000次迭代后采样生成的名字数量
    vocab_size -- 词汇表大小（即唯一字符数）
    
    返回值:
    parameters -- 训练完成后的参数字典（Waa、Wax、Wya、b、by）
    """
    
    # 输入和输出层的大小均等于词汇表大小
    n_x, n_y = vocab_size, vocab_size
    
    # ---------- 1. 初始化参数 ----------
    # Waa, Wax, Wya, b, by
    parameters = initialize_parameters(n_a, n_x, n_y)
    
    # ---------- 2. 初始化平滑损失 ----------
    # 目的：让损失值随时间平滑变化，避免剧烈波动
    loss = get_initial_loss(vocab_size, dino_names)
    
    # ---------- 3. 读取并预处理训练数据 ----------
    with open("dinos.txt") as f:
        examples = f.readlines()  # 每行一个恐龙名字
    # 转小写并去掉换行符
    examples = [x.lower().strip() for x in examples]
    
    # 随机打乱训练样本顺序，避免模型记住顺序模式
    shuffle(examples)
    
    # ---------- 4. 初始化隐藏状态 ----------
    a_prev = np.zeros((n_a, 1))
    
    # ---------- 5. 训练主循环 ----------
    for j in range(num_iterations):
        
        # 选择当前训练样本（循环使用）
        index = j % len(examples)
        # 输入序列 X，加上 None 作为起始标记
        X = [None] + [char_to_ix[ch] for ch in examples[index]]
        # 输出序列 Y = X 向左平移一位 + 换行符结尾
        Y = X[1:] + [char_to_ix["\n"]]
        
        # ---------- 6. 执行单步优化 ----------
        # 前向传播 -> 反向传播 -> 梯度修剪 -> 参数更新
        curr_loss, gradients, a_prev = optimize(X, Y, a_prev, parameters)
        
        # ---------- 7. 平滑损失 ----------
        # 防止损失曲线过于剧烈波动
        loss = smooth(loss, curr_loss)

        # ---------- 8. 每2000步打印一次样本 ----------
        if j % 2000 == 0:
            print('Iteration: %d, Loss: %f' % (j, loss) + '\n')
            
            seed = 0
            for name in range(dino_names):
                # 调用sample()函数根据当前模型生成新名字
                sampled_indexes = sample(parameters, char_to_ix, seed)
                print_sample(sampled_indexes, ix_to_char)
                seed += 1
            
            print('\n')
        
    return parameters


运行下一个代码单元，你会观察到模型在第一次迭代时输出随机字符。  
经过几千次迭代后，模型应该能够生成看起来比较合理的恐龙名字。


In [25]:
parameters = model(data, ix_to_char, char_to_ix)

Iteration: 0, Loss: 23.097228

Nkzxwtdmfqoeyhsqwasjjjvu
Kneb
Kzxwtdmfqoeyhsqwasjjjvu
Neb
Zxwtdmfqoeyhsqwasjjjvu
Eb
Xwtdmfqoeyhsqwasjjjvu


Iteration: 2000, Loss: 28.039557

Mivusalicoravhosocpineshansavieanaltailapgairatolo
Inca
Iwtos
Macaislhaclppthanwjedycerta
Xusbbidoravgoslariontoanthylganaltailapgairatolori
Ca
Tos


Iteration: 4000, Loss: 25.780069

Ngyusleomoraxatruf
Ingbakptia
Kytrojicholyiros
Ngaagosaurus
Xusndomosaurus
Da
Toraonos


Iteration: 6000, Loss: 24.739194

Nhytrodom
Klecalptia
Lyuspendor
Ngabersaurus
Xusrbilosaurus
Ee
Trodolosaurus


Iteration: 8000, Loss: 24.050955

Onxxtonnaseluptotasaurus
Llecalosaurus
Lyvspendopevdrotarbintochurynganenuomechiamofus
Olbaltopadrus
Xutolonosaurus
Eeakptia
Trohmeosaurus


Iteration: 10000, Loss: 23.646209

Onxusichisaurus
Jlacanosaurus
Kxusodon
Olaagrosaurus
Yusraonosaurus
Edalosaurus
Trodonisaurus


Iteration: 12000, Loss: 23.290685

Onyusodon
Licabasphacitesaurus
Lyuspendosaurus
Ohaaesticaptosaurus
Yuspendosaurus
Edalosaurus
Trodon

## 结论

你可以看到，随着训练的进行，你的算法开始生成看起来合理的恐龙名字。  
一开始，它生成的是随机字符，但到训练结束时，你可以看到一些有趣的恐龙名字结尾。  
你可以尝试运行更多迭代，并调整超参数，看看是否能得到更好的结果。  
我们的实现生成了一些非常酷的名字，如 `maconucon`、`marloralus` 和 `macingsersaurus`。  
你的模型也很可能学会了恐龙名字通常以 `saurus`、`don`、`aura`、`tor` 等结尾。

如果你的模型生成了一些不太酷的名字，也不要完全责怪模型——并不是所有真实的恐龙名字都听起来很酷。（例如，`dromaeosauroides` 就是真实的恐龙名字，并且在训练集中。）  
这个模型至少可以提供一组候选名字，从中挑选最酷的！

这个作业使用的数据集相对较小，因此你可以在 CPU 上快速训练 RNN。  
训练英文语言模型通常需要更大的数据集和更多的计算资源，并且可能需要在 GPU 上运行数小时。  
我们对恐龙名字模型训练了一段时间，到目前为止我们最喜欢的名字是：强大、无敌且凶猛的 Mangosaurus！

<img src="images/mangosaurus.jpeg" style="width:250;height:300px;">


## 4 - 像莎士比亚一样写作


一个类似的（但更复杂）任务是生成莎士比亚风格的诗歌。  
与恐龙名字不同，你可以使用莎士比亚诗歌集合作为数据集。  
利用 LSTM 单元，你可以学习跨越文本中许多字符的长期依赖关系——例如，序列中某个位置出现的字符可以影响很久之后应该出现的另一个字符。  
这些长期依赖在恐龙名字中不太重要，因为名字较短。

<img src="images/shakespeare.jpg" style="width:500;height:400px;">
<caption><center> 让我们成为诗人吧！ </center></caption>

我们已经使用Pytorch实现了一个莎士比亚诗歌生成器。  


我们使用的是莎士比亚诗歌集（名为 [*"The Sonnets"*](shakespeare.txt)）。


In [ ]:
# --------------------------------------------------------------
# 🌹 莎士比亚文本生成器（PyTorch）
# --------------------------------------------------------------

import torch
import torch.nn as nn
import numpy as np
import random
from torch.utils.data import DataLoader, TensorDataset

# ===============================
# 第1步：读取莎士比亚文本数据
# ===============================
print("加载莎士比亚文本中...")

# 你可以替换成任何英文语料文件，例如 "shakespeare.txt"
with open("shakespeare.txt", "r", encoding="utf-8") as f:
    text = f.read().lower()

# 取出文本中所有的唯一字符
chars = sorted(list(set(text)))
vocab_size = len(chars)

# 创建字符与索引的双向映射字典
char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}

print(f"字符总数: {vocab_size}")

# ===============================
# 第2步：创建训练数据集
# ===============================
print("创建训练集...")

Tx = 40  # 序列长度（每个样本的输入字符数）
stride = 3  # 每隔多少字符采样一次
X, Y = [], []

# 从文本中提取输入序列 X 和下一个字符 Y
for i in range(0, len(text) - Tx, stride):
    X.append([char_to_ix[ch] for ch in text[i: i + Tx]])
    Y.append(char_to_ix[text[i + Tx]])

print(f"训练样本数量: {len(X)}")

# ===============================
# 第3步：向量化训练集
# ===============================
print("向量化训练集...")

X_oh = np.zeros((len(X), Tx, vocab_size), dtype=np.float32)
Y_oh = np.zeros((len(X), vocab_size), dtype=np.float32)

for i, seq in enumerate(X):
    for t, ch_idx in enumerate(seq):
        X_oh[i, t, ch_idx] = 1
    Y_oh[i, Y[i]] = 1

# 转换为 PyTorch 张量
X_tensor = torch.tensor(X_oh)
Y_tensor = torch.tensor(Y_oh)

# 创建 DataLoader
dataset = TensorDataset(X_tensor, Y_tensor)
loader = DataLoader(dataset, batch_size=128, shuffle=True)

# ===============================
# 第4步：定义模型
# ===============================
class ShakespeareModel(nn.Module):
    def __init__(self, n_x, n_a=512):
        super(ShakespeareModel, self).__init__()
        self.lstm = nn.LSTM(input_size=n_x, hidden_size=n_a, batch_first=True)
        self.fc = nn.Linear(n_a, n_x)
        # ⚠️ 不添加 softmax，这样 CrossEntropyLoss 才能正确计算

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # 取序列最后一步输出
        return out  # 输出 logits（未归一化）

# ===============================
# 第5步：训练设置
# ===============================
print("开始训练...")

model = ShakespeareModel(vocab_size)
criterion = nn.CrossEntropyLoss()  # 自动包含 softmax
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

num_epochs = 30  # 训练轮数，可调大以提升生成质量

# ===============================
# 第6步：训练循环
# ===============================
for epoch in range(1, num_epochs + 1):
    total_loss = 0
    for batch_x, batch_y in loader:
        outputs = model(batch_x)
        loss = criterion(outputs, torch.argmax(batch_y, dim=1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch [{epoch}/{num_epochs}] Loss: {total_loss / len(loader):.6f}")

    # 每5轮保存一次模型并生成诗歌
    if epoch % 5 == 0:
        torch.save(model.state_dict(), "shakespeare_model.pth")
        print("💾 模型已保存。")

        # ===============================
        # 第7步：文本生成函数
        # ===============================
        def generate_text(seed, length=200):
            generated = seed
            for _ in range(length):
                # 构建输入
                x_pred = np.zeros((1, Tx, vocab_size))
                for t, ch in enumerate(seed[-Tx:]):
                    if ch in char_to_ix:
                        x_pred[0, t, char_to_ix[ch]] = 1.

                x_pred = torch.tensor(x_pred, dtype=torch.float32)

                # 预测下一个字符
                with torch.no_grad():
                    preds = torch.softmax(model(x_pred), dim=-1).cpu().numpy().ravel()

                # 根据预测概率采样
                next_idx = np.random.choice(range(vocab_size), p=preds)
                next_char = ix_to_char[next_idx]

                generated += next_char
                seed += next_char
            return generated

        # 生成一首诗
        print("\n🌹 生成的莎士比亚诗歌：\n")
        print(generate_text("a beautiful girl "))
        print("\n")

print("训练完成 ✅")


加载莎士比亚文本中...
字符总数: 38
创建训练集...
训练样本数量: 31412
向量化训练集...
开始训练...
Epoch [1/30] Loss: 2.973895
Epoch [2/30] Loss: 2.532324
Epoch [3/30] Loss: 2.324776
Epoch [4/30] Loss: 2.215475
Epoch [5/30] Loss: 2.133931
💾 模型已保存。

🌹 生成的莎士比亚诗歌：

a beautiful girl ldedsesaau sec n,nnopty cere paatule it the tay pfirgi l
cure,
whyt hon my whime batheerandif.

oref sorerot uy  alloth wiriln yaven the as mer
wht.


ou holss angeine pow,
tha ku dallog tor fritht an


Epoch [6/30] Loss: 2.055479
Epoch [7/30] Loss: 1.990296
Epoch [8/30] Loss: 1.927208
Epoch [9/30] Loss: 1.872323
Epoch [10/30] Loss: 1.819941
💾 模型已保存。

🌹 生成的莎士比亚诗歌：

a beautiful girl reapuderetnuo,diydbi
licinedabe,
tan unhis) chine our will dore resteint.


to
thin for the or ails aul frimmisu nove's,
or sealiofino:
ore bring i aurstoulling:
oud ingon my sllfis) but my tinten eye


Epoch [11/30] Loss: 1.761615
Epoch [12/30] Loss: 1.702048
Epoch [13/30] Loss: 1.638866
Epoch [14/30] Loss: 1.563454
Epoch [15/30] Loss: 1.482293
💾 模型已保存。

🌹 生成的莎士比亚诗歌：


RNN-莎士比亚模型与之前为恐龙名字构建的模型非常相似。唯一的主要区别有：

- 使用 LSTM 而不是基础 RNN，以捕捉更长的依赖关系  
- 模型更深，采用堆叠的 LSTM（2 层）  
- 使用 Keras 而不是纯 Python，以简化代码  

如果你想了解更多，也可以查看 Keras 团队在 GitHub 上的文本生成实现：[https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py](https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py)。

恭喜你完成了这个 Notebook！


**参考资料**：

- 本练习借鉴了 Andrej Karpathy 的实现：[https://gist.github.com/karpathy/d4dee566867f8291f086](https://gist.github.com/karpathy/d4dee566867f8291f086)。想了解更多关于文本生成的内容，也可以查看 Karpathy 的[博客文章](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)。  
- 关于莎士比亚诗歌生成器，我们的实现基于 Keras 团队的 LSTM 文本生成器实现：[https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py](https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py)。
